In [25]:
from __future__ import print_function
from rdkit import Chem
import tensorflow as tf
import numpy as np
import pandas as pd
import os
import gzip
import multiprocessing as mp

In [35]:
#get the 102 directors and extract the data 


path = "../DUDE/DUDE"
#get all folders
all_targ = os.listdir(path)
#output = mp.Queue()

#for 102 example..... get *sdf.gz


def make_dframe(name,rad=2,bi=2048):
    tmp_act = gzip.open(path+"/"+name+"/"+'actives_final.sdf.gz')
    tmp_dec = gzip.open(path+"/"+name+"/"+'decoys_final.sdf.gz')

        #read files
    cont_act = Chem.ForwardSDMolSupplier(tmp_act)
    cont_dec = Chem.ForwardSDMolSupplier(tmp_dec)

        #and make a list
    list_act = [x for x in cont_act if x is not None]
    list_dec = [x for x in cont_dec if x is not None]
    
    list_act = list_act[0:10]
    list_dec = list_dec[0:10]
    
    num_act = len(list_act)
    num_dec = len(list_dec)

    from rdkit.Chem import AllChem
    npMat = np.zeros((num_act+num_dec,bi+1))
    
    #add active/nonactive and name as num
   # npMat_y = np.zeros((num_act+num_dec,1))

    i = 0
    for m in list_act:
        npMat[i,0:-1] = [x for x in AllChem.GetMorganFingerprintAsBitVect(m,rad,nBits=bi)]
        npMat[i,-1] = 1
      #  npMat_y[i] = 1 
        i +=1
    for m in list_dec:
        npMat[i,0:-1] = [x for x in AllChem.GetMorganFingerprintAsBitVect(m,rad,nBits=bi)]
        npMat[i,-1] = 0
       # npMat_y[i] = 0
        i += 1
    return npMat#,npMat_y


first we want to load the data:

- ~/Documents/git/DUDE/DUDE contains a folder for each of the 102 targets in DUDE
- each folder contains *.ism / *.mol2 / *.sdf files for active and decoy molecules 
- to try to predict "bioactivity" (althouhg, how do we know that decoys are actually not active on other targers? <span style="color:red">check how exactly decoys are genereated</span>



In [42]:
pool = mp.Pool(2)

import time

start_t = time.time()
result = pool.map(make_dframe, [x for x in all_targ[0:2]]) 
end_t = time.time()
print(end_t-start_t)
pool.close()

18.740212202072144


In [ ]:
#get total dimention
totalDim = 0
for m in result:
    totalDim += m.shape[0]
    
#initiate and fill main matrix
npMat = np.zeros((totalDim,2048+1))
i = 0
for m in result:
        npMat[i:i+m.shape[0],:] = m
        i = m.shape[0]
        

In [77]:
feat = {}
i = 0
for col in npMat.T:
    feat[str(i)] = col
    i += 1
labels = npMat[:,-1]


In [90]:
def train_input_fn(features, labels, batch_size):
    """An input function for training"""
    # Convert the inputs to a Dataset.
    dataset = tf.data.Dataset.from_tensor_slices((features, labels))

    # Shuffle, repeat, and batch the examples.
    return dataset.shuffle(1000).repeat().batch(batch_size)

In [87]:
my_feature_columns = []
for key in feat.keys():
    my_feature_columns.append(tf.feature_column.numeric_column(key=key))

In [88]:
# Build a DNN with 2 hidden layers and 10 nodes in each hidden layer.
classifier = tf.estimator.DNNClassifier(
    feature_columns=my_feature_columns,
    # Two hidden layers of 10 nodes each.
    hidden_units=[10, 10],
    # The model must choose between 3 classes.
    n_classes=2)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': '/tmp/tmpdmzmi7x2', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7fba84a68fd0>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


In [91]:
classifier.train(
    input_fn=lambda:train_input_fn(feat,labels,100),
    steps=100)

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 1 into /tmp/tmpdmzmi7x2/model.ckpt.
INFO:tensorflow:loss = 70.02135, step = 0
INFO:tensorflow:Saving checkpoints for 100 into /tmp/tmpdmzmi7x2/model.ckpt.
INFO:tensorflow:Loss for final step: 0.05087628.
